In [1]:
import pyam
import pandas as pd

import dotenv
import os
from pathlib import Path

<IPython.core.display.Javascript object>

In [2]:
dotenv.load_dotenv()

True

Step 1: Read in the Gidden and Gasser et al. dataset

In [3]:
df = pyam.IamDataFrame(
    Path(
        os.environ['GIDDEN_DATA']
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file /Users/gauravganti/Documents/github_projects/2023_egr_ch7_equity/data/gidden_et_al_2022_ar6_reanalysis_data.csv


Step 2: Read in the Gidden and Gasser et al. metadata file

In [4]:
reanalysis_meta = pd.read_csv(
    Path(
        os.environ['GIDDEN_META']
    ),
    index_col=[0,1]
)

Step 3: Read in the Schleussner et al. metadata file

In [5]:
schleussner_meta = pd.read_excel(
    Path(
        os.environ['SCHLEUSSNER_META']
    ),
    index_col=[0,1]
)

Step 4: Downselect the following elements from the Schleussner et al metadata:
* The models and scenarios that are also present in the Gidden et al. metadata
* The column 'category_new_rolling_mean'

In [6]:
schleussner_meta_downselected = (
    schleussner_meta
    .loc[reanalysis_meta.index, 'category_new_rolling_mean']
)

Step 5: Assign this as an additional column to the Gidden et al. metadata

In [7]:
reanalysis_meta.loc[:, 'category_new_rolling_mean'] = schleussner_meta_downselected

Step 6: Assign this to the metadata of the dataset we loaded in Step 1.

In [8]:
df.meta = reanalysis_meta

Step 7: Filter out for the 1.5°C scenarios.

In [9]:
df_1p5 = (
    df
    .filter(
        Category=['C1','C2','C3']
    )
)

In [10]:
df_1p5.variable

['AR6 Reanalysis|OSCARv3.2|Carbon Removal',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Indirect',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only']

In [11]:
r5_data = pd.read_csv(
    os.environ['AR6_R5_DATA'],
    index_col=[0,1]
)

In [12]:
r5_data_selected = r5_data.loc[df_1p5.meta.index, :]

In [13]:
r5_data_selected_variables = r5_data_selected[
    r5_data_selected['Variable'].isin(
        [
            'Carbon Sequestration|CCS|Biomass',
            'Carbon Sequestration|Direct Air Capture',
            'Carbon Sequestration|Enhanced Weathering',
            'Population'
        ]
    ) 
].reset_index()

In [14]:
r5_pyam = pyam.IamDataFrame(r5_data_selected_variables)

In [15]:
r5_pyam.unit_mapping

{'Carbon Sequestration|CCS|Biomass': 'Mt CO2/yr',
 'Carbon Sequestration|Direct Air Capture': 'Mt CO2/yr',
 'Carbon Sequestration|Enhanced Weathering': 'Mt CO2/yr',
 'Population': 'million'}

In [16]:
r5_pyam_novel = r5_pyam.aggregate(
    variable='AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land',
    components=[
        'Carbon Sequestration|CCS|Biomass',
        'Carbon Sequestration|Direct Air Capture',
        'Carbon Sequestration|Enhanced Weathering'   
    ]
)

Now, we also pull out the `Population` variable that will be applied later in the assessment.

In [17]:
r5_pyam_population = r5_pyam.filter(
    variable='Population'
)

In [18]:
r5_pyam_population.interpolate(
    time=range(2020, 2101),
    inplace=True
)

In [19]:
r5_pyam_novel.interpolate(
    time=range(2020, 2101),
    inplace=True
)

In [20]:
df_1p5 = pyam.concat(
    [df_1p5, r5_pyam_novel, r5_pyam_population]
)

Step 8: Calculate the gross emission reductions.

In [21]:
df_1p5.aggregate(
    variable='AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total',
    components=[
        'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct',
        'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land'
    ],
    append=True
)

In [22]:
df_1p5.aggregate(
    variable='AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross',
    components=[
        'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only',
        'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total' # Checked with Matt
    ],
    append=True
)

Step 9: Convert all units to Gt

In [23]:
df_1p5.unit_mapping

{'AR6 Reanalysis|OSCARv3.2|Carbon Removal': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Indirect': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross': 'Mt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases': 'Mt CO2-equiv/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only': 'Mt CO2-equiv/yr',
 'Population': 'million'}

In [24]:
df_1p5.convert_unit(
    current='Mt CO2/yr',
    to='Gt CO2/yr',
    inplace=True
)

In [25]:
df_1p5.convert_unit(
    current='Mt CO2-equiv/yr',
    to='Gt CO2-equiv/yr',
    inplace=True
)

In [26]:
df_1p5.unit_mapping

{'AR6 Reanalysis|OSCARv3.2|Carbon Removal': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Indirect': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross': 'Gt CO2/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases': 'Gt CO2-equiv/yr',
 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only': 'Gt CO2-equiv/yr',
 'Population': 'million'}

In [27]:
column_net_zero = 'Year of netzero CO2 emissions (Harm-Infilled) table'

Step 14: We will need to calculate the cumulative removals between net zero and 2100

In [28]:
cdr_total = (
    df_1p5
    .filter(
        variable='*Total',
        region='World'
    )
    .timeseries()
)

In [29]:
cdr_cumulative_net_zero_2100 = cdr_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero],
        last_year=2100
    ),
    axis=1
)

In [30]:
df_1p5.set_meta(
    meta=cdr_cumulative_net_zero_2100,
    name='cumulative_cdr_netzero_2100'
)

Step 15: Set the gross emissions and total CDR at net zero across all the pathways

In [31]:
cdr_net_zero = cdr_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero],
        last_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero]
    ),
    axis=1
)

In [32]:
df_1p5.set_meta(
    meta=cdr_net_zero,
    name='cdr_net_zero'
)

In [33]:
gross_total = (
    df_1p5
    .filter(
        variable='*Gross',
        region='World'
    )
    .timeseries()
)

In [34]:
gross_net_zero = gross_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero],
        last_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero]
    ),
    axis=1
)

In [35]:
df_1p5.set_meta(
    meta=gross_net_zero,
    name='gross_net_zero'
)

In [36]:
gross_cumulative = gross_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero],
        last_year=2100
    ),
    axis=1
)

In [37]:
gross_cumulative_2020 = gross_total.apply(
    lambda x: pyam.cumulative(
        x,
        first_year=2020,
        last_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero]
    ),
    axis=1
)

In [38]:
df_1p5.set_meta(
    meta=gross_cumulative,
    name='cumulative_gross_netzero_2100'
)

In [39]:
df_1p5.set_meta(
    meta=gross_cumulative_2020,
    name='cumulative_gross_2020_netzero'
)

Step 15: For the land- and novel CDR, calculate the cumulative removal between 2020 and net zero CO2.

In [40]:
to_crunch = [
    ('land_cdr_2020_netzero', 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct'),
    ('novel_cdr_2020_netzero', 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land')
]

In [41]:
for col, var in to_crunch:
    data_ts = (
        df_1p5
        .filter(
            variable=var,
            region='World'
        )
        .timeseries()
    )
    df_1p5.set_meta(
        data_ts.apply(
            lambda x: pyam.cumulative(
                x,
                first_year=2020,
                last_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero]
            ),
            axis=1
        ),
        name=col
    )

In [42]:
to_crunch = [
    ('land_cdr_netzero_2100', 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct'),
    ('novel_cdr_netzero_2100', 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Non-Land')
]

In [43]:
for col, var in to_crunch:
    data_ts = (
        df_1p5
        .filter(
            variable=var,
            region='World'
        )
        .timeseries()
    )
    df_1p5.set_meta(
        data_ts.apply(
            lambda x: pyam.cumulative(
                x,
                first_year=df_1p5.meta.loc[(x.name[0], x.name[1]), column_net_zero],
                last_year=2100
            ),
            axis=1
        ),
        name=col
    )

In [44]:
final_data = df_1p5

Step 15: Save this data out for further analysis.

In [45]:
final_data.to_excel(
    Path(
        '../data/101_data.xlsx'
    )
)